#### Importing the relevant libraries

In [1]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import IntegerType
from dataiku.snowpark import DkuSnowpark
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark import version
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import when, col, lit, avg, stddev, stddev_pop , round, log

### Creating the session

In [ ]:
dku_snowpark = DkuSnowpark()

### Reading the datasets

In [ ]:
LOANS = dataiku.Dataset("LOANS_ENRICHED_joined_prepared")
snowdf = dku_snowpark.get_dataframe(LOANS)

In [ ]:
snowdf.count()

In [ ]:
snowdf.show(2)

### To pandas

In [ ]:
snowdf_pandas=snowdf.toPandas()
snowdf_pandas.head(5)

### Lets do Feature Engineering using SnowPark

### We will create a new column EMP_BUCKET,  select top 5 Emp Title and rest NA

In [ ]:
snowdf_pandas['EMP_TITLE'].value_counts()

In [ ]:
snowdf = snowdf.withColumn('EMP_BUCKET',
                               when(snowdf.EMP_TITLE == 'Teacher','Teacher' )
                               .when(snowdf.EMP_TITLE == 'Manager', 'Manager')
                              .when(snowdf.EMP_TITLE == 'Registered Nurse', 'Registered Nurse')
                           .when(snowdf.EMP_TITLE == 'Driver', 'Driver')
                            .when(snowdf.EMP_TITLE == 'Owner', 'Owner')
                               .otherwise('NA'))

# Select the first set of columns
snowdf.select("EMP_TITLE","EMP_BUCKET").filter(snowdf.EMP_BUCKET == 'NA').show()

In [ ]:
# Select the first set of columns
snowdf.select("EMP_TITLE","EMP_BUCKET").filter(snowdf.EMP_BUCKET  != 'NA').show()

### Scaling INSTALLMENT column by creating INSTALL_NORM column

In [ ]:
# Define max and min values and collect them

mean_installement = snowdf.agg({'INSTALLMENT': 'mean'}).collect()[0][0]
stddev_installement = snowdf.agg({'INSTALLMENT': 'stddev'}).collect()[0][0]

# Create a new column based off the scaled data

snowdf = snowdf.withColumn('INSTALL_NORM',
                  ((snowdf['INSTALLMENT'] - mean_installement) / stddev_installement))

snowdf.select("INSTALLMENT","INSTALL_NORM").show()

### Count Missing rows

In [ ]:
# Count missing rows
missing_emp_length_years = snowdf.where(snowdf['EMP_LENGTH_YEARS'].isNull()).count()
missing_emp_length_years

### Imputing median value

In [ ]:
# Calculate the median value

col_median = int(snowdf.agg({'EMP_LENGTH_YEARS': 'median'}).collect()[0][0])

# Replacing with the median value for that column
snowdf=snowdf.na.fill({'EMP_LENGTH_YEARS': col_median})
snowdf.select("EMP_LENGTH_YEARS").show()

In [ ]:
# Count missing rows
missing_emp_length_years = snowdf.where(snowdf['EMP_LENGTH_YEARS'].isNull()).count()
missing_emp_length_years

### Dropping these columns 'EMP_TITLE','INSTALLMENT' as we will be using new columns for ML

In [ ]:
# Drop the columns
snowdf = snowdf.drop('INSTALLMENT')

In [ ]:
snowdf.select("INSTALL_NORM","EMP_LENGTH_YEARS","EMP_BUCKET").show()

In [ ]:
# get output dataset
LOANS_FE = dataiku.Dataset("LOANS_FE")

# write input dataframe to output dataset
dku_snowpark.write_with_schema(LOANS_FE,snowdf)